# dev_equity_builder
Записываем шаги для build_equity_v2: сначала лог и timeline ±5 минут, потом outer merge, затем объединение с свечами и очистка, а после — вычисление финальных цен и equity.

In [ ]:
from pathlib import Path

import pandas as pd

from src.data.csv_transformer_profit import load_transform_csv

pd.set_option("display.max_rows", 400)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

log_path = Path("trade_data/T_GLD_new_tr_1t.csv")
trades = load_transform_csv(log_path)
trades["date_time"] = trades["date_time"].dt.floor("min")
trades["position_profit"] = trades.get("position_profit", pd.Series(0.0, index=trades.index)).fillna(0.0)
print("trades head")
display(trades.head(5))

In [ ]:
start = trades["date_time"].min() - pd.Timedelta("5min")
end = trades["date_time"].max() + pd.Timedelta("5min")
timeline = pd.date_range(start, end, freq="1min")
frame = pd.DataFrame({"date_time": timeline})
print("frame length", len(frame))

In [ ]:
outer_merge = frame.merge(trades, on="date_time", how="outer")
outer_merge = outer_merge.sort_values("date_time").reset_index(drop=True)
print("outer join result -- строки:", len(outer_merge))
display(outer_merge.head(10))

In [ ]:
candles = pd.read_csv("OHCL/T_GLD_new_tr_1t_ohcl.csv", parse_dates=["DATE_TIME"])
if "date_time" not in candles.columns:
    candles["date_time"] = candles["DATE_TIME"].dt.floor("min")
candles = candles.sort_values("date_time")
frame_with_candles = frame.merge(candles, on="date_time", how="left", suffixes=("", "_candle"))
frame_with_candles = frame_with_candles.merge(trades, on="date_time", how="left", suffixes=("", "_trade"))
mask_empty = frame_with_candles["price"].isna() & frame_with_candles["current_pos"].isna() & frame_with_candles["position_profit"].isna() & frame_with_candles["CLOSE"].isna()
frame_with_candles = frame_with_candles.loc[~mask_empty].reset_index(drop=True)
print("связка frame+candles+trades (без пустых)", len(frame_with_candles))
display(frame_with_candles.head(10))

In [ ]:
merged = frame_with_candles.copy()
merged["price_from_log"] = merged["price"]
merged["final_price"] = merged["price_from_log"].fillna(merged["CLOSE"])
merged["price_source"] = merged["price_from_log"].notna()
print("первая часть цен")
display(merged[["date_time", "price_from_log", "final_price", "price_source"]].head(10))print("пустые строки внутри периода")
display(merged.loc[merged["price_from_log"].isna()].head(5))

In [ ]:
merged["current_pos"] = merged["current_pos"].ffill().fillna(0).astype(int)
merged["position_profit"] = merged["position_profit"].fillna(0.0)
merged["prev_position"] = merged["current_pos"].shift(1).fillna(0).astype(int)
merged["action_price"] = merged["final_price"]
position_changed = merged["current_pos"] != merged["prev_position"]
merged["deal_price"] = merged["action_price"].where(position_changed)
merged["prev_action_price"] = merged["action_price"].shift(1)
merged["delta_price"] = merged["action_price"] - merged["prev_action_price"]
step = merged["1_step_price"].fillna(method="ffill")
merged["comis_count"] = (merged["current_pos"] - merged["prev_position"]).abs() * 0.0
merged["Pnl"] = merged["prev_position"] * merged["delta_price"] * step
merged["Equity"] = merged["Pnl"].fillna(0).cumsum()
merged["bot_equity"] = merged["position_profit"].cumsum()
print("результирующий участок equity")
display(merged[["date_time", "current_pos", "deal_price", "final_price", "Equity", "bot_equity"]].head(10))print("tail")
display(merged[["date_time", "current_pos", "final_price", "Equity", "bot_equity"]].tail(5))